In [2]:
# import the necessary packages
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import scipy.stats as stats
import math

import numpy as np

import seaborn as sns
import sklearn

pd.set_option('display.max_columns', 90) # by default 10, pandas will display all columns
pd.set_option('display.max_rows', 15) # by default 10, pandas will display all rows

In [3]:
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer

import pickle

from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline

from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier

from sklearn import metrics

from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split

from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_score,recall_score
from sklearn.metrics import plot_confusion_matrix

In [4]:
#read the test and train data
# 1.Upload data
train = pd.read_csv('/Users/coraliaarguedasf/Documents/BootcampDataScience/Week14/train.csv')
#test = pd.read_csv('/Users/coraliaarguedasf/Documents/BootcampDataScience/Week14/test.csv')
#review the shape
train = train[~train['X2'].str.contains('SEX')]
print(train.shape)
#print(test.shape)


(23999, 25)


In [5]:
train.drop('Unnamed: 0',axis=1,inplace=True)
train.head()

X1 X2 X3 X4  X5 X6 X7  X8  X9 X10 X11     X12     X13     X14     X15  \
0   50000  2  2  2  26  1  2   2   0   0   0   50867   51486   48869   30115   
1  180000  2  5  1  44  0  0  -1  -1  -1  -1   20916       0     850       0   
2  270000  2  2  1  56  0  0   0   0   0   0  137807  123769  104377  106651   
3   80000  2  2  2  25  1  2   0   0   0   0   78894   79989   79882   67946   
4  100000  2  2  1  28  2  0   0   0   0   2   73073   74739   70844   63924   

      X16     X17   X18   X19   X20   X21    X22   X23  Y  
0   29527   29998  1710   136  1206  2002   2005  3003  0  
1    6881   10340     0   850     0  6881  10340   182  0  
2  109294  101151  5000  5000  3953  4500   4000  3900  0  
3   51435   53113  3000  3500  5000  2500   2500  2000  0  
4   57326   59654  3500  3003  1910  2400   3300     0  1

In [142]:
pd.options.display.max_rows =None
train.dtypes

X1     object
X2     object
X3     object
X4     object
X5     object
X6     object
X7     object
X8     object
X9     object
X10    object
X11    object
X12    object
X13    object
X14    object
X15    object
X16    object
X17    object
X18    object
X19    object
X20    object
X21    object
X22    object
X23    object
Y      object
dtype: object

In [143]:
train.isnull().sum()

X1     0
X2     0
X3     0
X4     0
X5     0
X6     0
X7     0
X8     0
X9     0
X10    0
X11    0
X12    0
X13    0
X14    0
X15    0
X16    0
X17    0
X18    0
X19    0
X20    0
X21    0
X22    0
X23    0
Y      0
dtype: int64

In [144]:
train.columns

Index(['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11',
       'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21',
       'X22', 'X23', 'Y'],
      dtype='object')

In [6]:
d = train.select_dtypes(include=['object']).columns
train[d] = train[d].astype(np.int16)

In [7]:
#Baseline
train['Y'].value_counts()

0    18691
1     5308
Name: Y, dtype: int64

In [8]:
X = train.drop('Y',axis=1)
y = train['Y']
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#X_Real = test.drop('Y') #test for kaggle

print(y_test.head())

3111     0
19275    0
17472    0
19921    0
20800    0
Name: Y, dtype: int16


In [9]:
names_numeric=x_train.describe().columns
#names_categorical=x_train.describe(exclude='number').columns

In [10]:
# 3.Data Preprocessing LinearRegression()
## Pipeline auxiliary function work with the different data types
numerical = names_numeric
#Ordinal Encoder
#ordinal_p = Pipeline([
    #("imputer", SimpleImputer(strategy='most_frequent')),
    #("encoder", OrdinalEncoder()),
#])
#Nominal Encoder
#nominal_p = Pipeline([
    #("imputer", SimpleImputer(strategy='most_frequent')),
    #("encoder", OneHotEncoder(handle_unknown='ignore'))
#])
#Numerical Scaling
numerical_p = Pipeline([
    ("imputer" , SimpleImputer(strategy='mean')), #median is the default strategy we can change to mean
    ("scaler", StandardScaler())
])

#Final Pipeline
preprocessing_pipeline = ColumnTransformer([
    #("nominal_prep", nominal_p, nominal),
    #("ordinal_prep", ordinal_p, ordinal),
    ("numerical_prep", numerical_p, numerical)
])

Logistic Regression

In [150]:
#Complete Pipeline
ML_model = LogisticRegression()

complete_pipeline_linear = Pipeline([('preprocessor',preprocessing_pipeline), ('model',ML_model)])

In [151]:
#GridSearch
# Create param grid to hyper parameter tune the model
param_grid = {
    "preprocessor__numerical_prep__imputer__strategy": ["mean", "median", "most_frequent"], # adjust the strategy for imputation
    'model__C':[1,0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1],
    'model__max_iter':[1,0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1],
}

grid_linear = GridSearchCV(complete_pipeline_linear, # Model
                    param_grid=param_grid, # Parameters to tune
                    cv=5, # Cross Validation
                    verbose=2, # Shows output while training 
                    n_jobs=-1 # How many core to use on your computer (-1 means use all cores)
                   )


In [152]:
#4. Prediction and Evaluation

grid_linear.fit(x_train, y_train)

y_pred = grid_linear.best_estimator_.predict(x_test)

print('Train')
print('Accuracy:',grid_linear.best_estimator_.score(x_train,y_train))

print('Test')
print('Accuracy:',grid_linear.best_estimator_.score(x_test,y_test))
print('Precision:',precision_score(y_test, y_pred, average='binary'))
print('Recall:',recall_score(y_test, y_pred, average='binary'))

Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Train
Accuracy: 0.78936402937653
Test
Accuracy: 0.7995833333333333
Precision: 0.5499474237644585
Recall: 0.49479659413434246


/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Random Forest

In [153]:
#Complete Pipeline
ML_model = RandomForestClassifier(n_estimators=500, max_depth=9, random_state=0)
complete_pipeline_random = Pipeline([('preprocessor',preprocessing_pipeline), ('model',ML_model)])

In [154]:
# Categorical
param_grid2 = {
    "model__n_estimators": [5,6,10,20,30,60,90,100,300,500], # how many trees in our forest
    "model__max_depth": [2,4,6]
    
}
grid_random = GridSearchCV(
    complete_pipeline_random ,
    param_grid2,
    cv = 5,
    n_jobs=-1,
    verbose=1)

In [155]:
#4. Prediction and Evaluation 

grid_random.fit(x_train, y_train)

y_pred = grid_random.best_estimator_.predict(x_test)

print('Train')
print('Accuracy:',grid_random.best_estimator_.score(x_train,y_train))

print('Test')
print('Accuracy:',grid_random.best_estimator_.score(x_test,y_test))
print('Precision:',precision_score(y_test, y_pred, average='binary'))
print('Recall:',recall_score(y_test, y_pred, average='binary'))

Fitting 5 folds for each of 30 candidates, totalling 150 fits
Train
Accuracy: 0.8170738059273921
Test
Accuracy: 0.8195833333333333
Precision: 0.6840077071290944
Recall: 0.3358561967833491


ExtraTreesClassifier

In [156]:
#Complete Pipeline
ML_model = ExtraTreesClassifier()

complete_pipeline_erandom = Pipeline([('preprocessor',preprocessing_pipeline), ('model',ML_model)])

In [157]:
# Categorical
param_grid2 = {
    "model__n_estimators": [5,6,10,20,30,60,90,100,300,500], # how many trees in our forest
    "model__max_depth": [2,4,6]
    
}
grid_erandom = GridSearchCV(
    complete_pipeline_erandom ,
    param_grid2,
    cv = 5,
    n_jobs=-1,
    verbose=1)

In [158]:
#4. Prediction and Evaluation 

grid_erandom.fit(x_train, y_train)

y_pred = grid_erandom.best_estimator_.predict(x_test)

print('Train')
print('Accuracy:',grid_erandom.best_estimator_.score(x_train,y_train))

print('Test')
print('Accuracy:',grid_erandom.best_estimator_.score(x_test,y_test))
print('Precision:',precision_score(y_test, y_pred, average='binary'))
print('Recall:',recall_score(y_test, y_pred, average='binary'))

Fitting 5 folds for each of 30 candidates, totalling 150 fits
Train
Accuracy: 0.8170738059273921
Test
Accuracy: 0.8195833333333333
Precision: 0.6891891891891891
Recall: 0.19299905392620625


Ridge

In [159]:
#Complete Pipeline
ML_model = RidgeClassifier(alpha=20)
complete_pipeline_ridge = Pipeline([('preprocessor',preprocessing_pipeline), ('model',ML_model)])

In [160]:
param_grid4 = {
    "model__alpha":[20, 25, 30,40,60,100], # how many trees in our forest
}
grid_ridge = GridSearchCV(
    complete_pipeline_ridge,
    param_grid4,
    cv = 5,
    n_jobs=-1,
    verbose=1)

In [161]:
#4. Prediction and Evaluation 

grid_ridge.fit(x_train, y_train)

y_pred = grid_ridge.best_estimator_.predict(x_test)

print('Train')
print('Accuracy:',grid_ridge.best_estimator_.score(x_train,y_train))

print('Test')
print('Accuracy:',grid_ridge.best_estimator_.score(x_test,y_test))
print('Precision:',precision_score(y_test, y_pred, average='binary'))
print('Recall:',recall_score(y_test, y_pred, average='binary'))

Fitting 5 folds for each of 6 candidates, totalling 30 fits
Train
Accuracy: 0.7950934944528361
Test
Accuracy: 0.800625
Precision: 0.7688172043010753
Recall: 0.13528855250709557


SVM

In [162]:
#Complete Pipeline
ML_model = SVC(kernel = 'linear')

complete_pipeline_svm = Pipeline([('preprocessor',preprocessing_pipeline), ('model',ML_model)])

In [163]:
#GridSearch
# Create param grid to hyper parameter tune the model
param_grid = {
    "preprocessor__numerical_prep__imputer__strategy": ["mean", "median", "most_frequent"], # adjust the strategy for imputation
}

grid_svm = GridSearchCV(complete_pipeline_svm, # Model
                    param_grid=param_grid, # Parameters to tune
                    cv=5, # Cross Validation
                    verbose=2, # Shows output while training 
                    n_jobs=-1 # How many core to use on your computer (-1 means use all cores)
                   )


In [164]:
#4. Prediction and Evaluation Linear Regression

grid_svm.fit(x_train, y_train)

y_pred = grid_svm.best_estimator_.predict(x_test)

print('Train')
print('Accuracy:',grid_svm.best_estimator_.score(x_train,y_train))

print('Test')
print('Accuracy:',grid_svm.best_estimator_.score(x_test,y_test))
print('Precision:',precision_score(y_test, y_pred, average='binary'))
print('Recall:',recall_score(y_test, y_pred, average='binary'))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Train
Accuracy: 0.8079066618053024
Test
Accuracy: 0.8085416666666667
Precision: 0.6864864864864865
Recall: 0.2403027436140019


PCA

In [165]:
#Complete Pipeline
ML_model = LogisticRegression()

complete_pipeline_linear_P = Pipeline([('preprocessor',preprocessing_pipeline),('pca',PCA()),('model',ML_model)])

In [166]:
#GridSearch
# Create param grid to hyper parameter tune the model
param_grid = {
    "preprocessor__numerical_prep__imputer__strategy": ["mean", "median", "most_frequent"], # adjust the strategy for imputation
    'pca__n_components':[1,2,3,4,5,6,7,8,9,10],
    'model__C':[1,0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1],
    'model__max_iter':[1,0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1],

}

grid_linear_p= GridSearchCV(complete_pipeline_linear_P, # Model
                    param_grid=param_grid, # Parameters to tune
                    cv=5, # Cross Validation
                    verbose=2, # Shows output while training 
                    n_jobs=-1, # How many core to use on your computer (-1 means use all cores)
                    scoring="accuracy"
                   )


In [167]:
#4. Prediction and Evaluation 

grid_linear_p.fit(x_train, y_train)

y_pred = grid_linear_p.best_estimator_.predict(x_test)

print('Train')
print('Accuracy:',grid_linear_p.best_estimator_.score(x_train,y_train))

print('Test')
print('Accuracy:',grid_linear_p.best_estimator_.score(x_test,y_test))
print('Precision:',precision_score(y_test, y_pred, average='binary'))
print('Recall:',recall_score(y_test, y_pred, average='binary'))

Fitting 5 folds for each of 3000 candidates, totalling 15000 fits
Train
Accuracy: 0.7844679410385957
Test
Accuracy: 0.7954166666666667
Precision: 0.5414364640883977
Recall: 0.46357615894039733


/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Stacking Ensemble

In [11]:
# define the base models
level0 = list()
level0.append(('lr', LogisticRegression(solver='liblinear')))
level0.append(('knn', KNeighborsClassifier()))
level0.append(('svm', SVC()))
level0.append(('bayes', GaussianNB()))
# define meta learner model
level1 = LogisticRegression(solver='liblinear')
# define the stacking ensemble
model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)
# fit the model on all available data
complete_pipeline = Pipeline(steps=[('preprocessor',preprocessing_pipeline),('model',model)])
complete_pipeline.fit(x_train, y_train)
y_pred=complete_pipeline.predict(x_test)
print('Train')
print('Accuracy:',complete_pipeline.score(x_train,y_train))

print('Test')
print('Accuracy:',complete_pipeline.score(x_test,y_test))
print('Precision:',precision_score(y_test, y_pred, average='binary'))
print('Recall:',recall_score(y_test, y_pred, average='binary'))

Train
Accuracy: 0.824105422157404
Test
Accuracy: 0.8214583333333333
Precision: 0.6736111111111112
Recall: 0.36707663197729423


The Chosen Model 

In [19]:
# define the base models
level0 = list()
level0.append(('lr', LogisticRegression(solver='liblinear')))
level0.append(('knn', KNeighborsClassifier()))
level0.append(('svm', SVC()))
level0.append(('bayes', GaussianNB()))
# define meta learner model
level1 = LogisticRegression(solver='liblinear')
# define the stacking ensemble
complete = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)
complete.fit(x_train, y_train)


StackingClassifier(cv=5,
                   estimators=[('lr', LogisticRegression(solver='liblinear')),
                               ('knn', KNeighborsClassifier()), ('svm', SVC()),
                               ('bayes', GaussianNB())],
                   final_estimator=LogisticRegression(solver='liblinear'))

Results

Logistic Regression:
Train
Accuracy: 0.78936402937653
Test
Accuracy: 0.7995833333333333
Precision: 0.5499474237644585
Recall: 0.49479659413434246
_____________________________
Random Forest:
Train
Accuracy: 0.8170738059273921
Test
Accuracy: 0.8195833333333333
Precision: 0.6840077071290944
Recall: 0.3358561967833491
______________________________
ExtraTreesClassifier:
Train
Accuracy: 0.8170738059273921
Test
Accuracy: 0.8195833333333333
Precision: 0.6891891891891891
Recall: 0.19299905392620625
______________________________
Ridge:
Train
Accuracy: 0.7950934944528361
Test
Accuracy: 0.800625
Precision: 0.7688172043010753
Recall: 0.13528855250709557
______________________________
SVM:
Train
Accuracy: 0.8079066618053024
Test
Accuracy: 0.8085416666666667
Precision: 0.6864864864864865
Recall: 0.2403027436140019
_____________________________
PCA:
Train
Accuracy: 0.7844679410385957
Test
Accuracy: 0.7954166666666667
Precision: 0.5414364640883977
Recall: 0.46357615894039733
_____________________________
Stacking Ensemble:
Accuracy: 0.824105422157404
Test
Accuracy: 0.8214583333333333
Precision: 0.6736111111111112
Recall: 0.36707663197729423

In [22]:
#Create a file with a model
pickle.dump(complete,open('classification_model.pkl','wb'))